# Decay Test

In this notebook, we will go through a nuclear decay test by hand to test the TARDIS decay calculation from pyne and the TARDIS model loader for the CSVY model type.

In [1]:
import numpy as np
import astropy.units as u

from tardis.io.config_reader import Configuration
from tardis.model import Radial1DModel

/Users/marcwilliamson/miniconda3/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


## Load the csvy model

In [2]:
tardis_config = Configuration.from_yaml('config_csvy_full.yml')

In [3]:
csvy_model = Radial1DModel.from_csvy(tardis_config)

### First we inspect the raw abundance and isotope_abundances specified by the user. This does not include decay.

In [4]:
csvy_model.raw_abundance

,0,1
atomic_number,,
1,0.00,0.40
2,0.99,0.58


In [5]:
csvy_model.raw_isotope_abundance

,,0,1
atomic_number,mass_number,,
28,56,0.01,0.02


### Now we inspect the calculated abundance including decay. Note below that the time of explosion is 16 days and the isotope abundances were defined at day 0.

In [6]:
csvy_model.time_explosion.to('day')

<Quantity 16. d>

In [7]:
csvy_model.abundance

,0,1
atomic_number,,
1,0.000000,0.400000
2,0.990000,0.580000
26,0.000736,0.001471
27,0.007653,0.015306
28,0.001611,0.003223


## Nuclear Decay

In general, the differential equations determining nuclear decay are complex, but there are equations called the Bateman Equations that handle the calculation.  Please see the Pyne documentation (https://pyne.io/theorymanual/decay.html) and this dissertation for more detailed discussion (https://apps.dtic.mil/dtic/tr/fulltext/u2/a469273.pdf).

For the current example, we have a very simple case of nuclear decay for the chain $^{56}Ni\rightarrow^{56}Co\rightarrow^{56}Fe$ without any branching. Let us use the following definitions:

$N1$ is the Nickel abundance at time $t$

$N2$ is the Cobalt abundance at time $t$

For this example, since $^{56}Fe$ is stable, we will say that the Fe abundance is equal to $N1(0)-N1(t)-N2(t)$.



The Bateman Equation for this simple example tells us...

$N1(t=N1(0)e^{-\lambda_{1}t}$

and...

$N2(t)=\lambda_{1}N1(0)\left[\frac{e^(-\lambda_{1}t)}{(\lambda_{2}-\lambda_{1})} + \frac{e^{-\lambda_{2}t}}{(\lambda_{1}-\lambda_{2})}\right]$

In [8]:
Ni_halflife = 6.075 * u.d
Co_halflife = 77.233 * u.d

lambda_Ni = np.log(2)/Ni_halflife
lambda_Co = np.log(2)/Co_halflife


def N1(t, N0, lambda1):
    return N0*np.exp(-lambda1*t)

def N2(t, N1_0, lambda_1, lambda_2):
    return lambda_1*N1_0*(np.exp(-lambda_1*t)/(lambda_2-lambda_1) +
                          np.exp(-lambda_2*t)/(lambda_1-lambda_2))

In [9]:
# This should be the Ni abundance in the first shell of the
# CSVY model after decay.
N1(16*u.d, 0.01, lambda_Ni)

<Quantity 0.00161125>

In [10]:
# This should be the Co abundance in the first shell of the
# CSVY model after decay.
N2(16*u.d, 0.01, lambda_Ni, lambda_Co)

<Quantity 0.0076531>

In [11]:
# This should be the Fe abundance in the first shell of the 
# CSVY model after decay.
.01-N1(16*u.d, 0.01, lambda_Ni)-N2(16*u.d, 0.01, lambda_Ni, lambda_Co)

<Quantity 0.00073565>